In [1]:
from readlif.reader import LifFile
import numpy as np
from tifffile import imwrite
import os
import pandas as pd
import numpy as np
import tifffile
import re

# type lif path, will display all stack names
# type desired stack name, if correct will display metadata
lif_path = r"D:\20240531_802confocal_human samples\20240531 3D HnE human samples001.lif"
stack_name = 'Mark_and_Find 003/Position003'
safe_name = re.sub(r'[<>:"/\\|?*]', '', stack_name)
out_folder = r"C:\Users\bear\Desktop\figure 3"
lif = LifFile(lif_path)

match = 0
for idx, img in enumerate(lif.image_list):
    print(img['name'])
    if img['name'] == stack_name:
        stack = img
        stack_idx = idx
        match = 1
print()

if match == 1:
    for key, value in stack.items():
        if key != 'settings':
            print(f"{key}: {value}")

Mark_and_Find 001/Position001
Mark_and_Find 002/Position003
Mark_and_Find 003/Position003
Mark_and_Find 003/Position004
Mark_and_Find 003/Position005
Mark_and_Find 003/Position006

dims: Dims(x=4496, y=4496, z=543, t=1, m=1)
display_dims: (1, 2)
dims_n: {1: 4496, 2: 4496, 3: 543}
scale_n: {1: 11.6, 2: 11.6, 3: 2.8846634896338395}
path: 20240531 3D HnE human samples001.xlef/Mark_and_Find 003/
name: Mark_and_Find 003/Position003
channels: 2
scale: (11.6, 11.6, 2.8846634896338395, None)
bit_depth: (16, 16)
mosaic_position: []
channel_as_second_dim: False


In [41]:
# store lif file name, stack name, xy pixel size and z pixel size in csv
# not automatic to loop over all files, instead use it when needed
metadata = {
    'folder': os.path.basename(os.path.dirname(lif_path)),
    'lif_name': os.path.basename(lif_path),
    'stack_name': stack_name,
    'num_channels': stack['channels'],
    'pixel_size_xy': 1/stack['scale'][0],  # microns/pixel
    'pixel_size_z': abs(1/stack['scale'][2]),   # microns/pixel
    'bit_depth': stack['bit_depth'][0]
}

csv_path = 'metadata.csv'
if os.path.exists(csv_path):
    df_existing = pd.read_csv(csv_path)
    df_new = pd.concat([df_existing, pd.DataFrame([metadata])], ignore_index=True)
    df_new = df_new.drop_duplicates(subset=['folder', 'lif_name', 'stack_name'], keep='last')
    df_new.to_csv(csv_path, index=False)
else:
    pd.DataFrame([metadata]).to_csv(csv_path, index=False)

In [42]:
# output tif (multichannel)
safe_name = re.sub(r'[<>:"/\\|?*]', '', stack_name)
tif_name = f"{safe_name}.tif"
tif_path = os.path.join(out_folder, tif_name)
stack_img = lif.get_image(stack_idx)
num_z, num_c, num_y, num_x, bit_depth = stack_img.dims.z, stack_img.channels, stack_img.dims.y, stack_img.dims.x, stack_img.bit_depth[0]
if bit_depth == 8:
    dtype = np.uint8
elif bit_depth == 16:
    dtype = np.uint16
    
tif_stack = np.zeros((num_z, num_c, num_y, num_x), dtype=dtype)
for z in range(num_z):
    for c in range(num_c):
        tif_stack[z, c] = stack_img.get_frame(z=z, t=0, c=c)
    if (z + 1) % 50 == 0:
        print(f'plane {z + 1} finished')

imwrite(tif_path, tif_stack, imagej = True, metadata = {'axes': 'ZCYX'})

plane 50 finished
plane 100 finished
plane 150 finished
plane 200 finished
plane 250 finished
plane 300 finished
plane 350 finished
plane 400 finished
plane 450 finished
plane 500 finished


C:\Users\bear\AppData\Local\Programs\Python\Python312\Lib\site-packages\tifffile\tifffile.py:3784: UserWarning: <tifffile.TiffWriter 'Mark_and_Find 003Position003.tif'> truncating ImageJ file
  warnings.warn(


In [2]:
# reuse python matlab to convert to single channel
import os
import tifffile

multichannel_path = os.path.join(out_folder, f"{safe_name}.tif")

folder, filename = os.path.split(multichannel_path)
base, ext = os.path.splitext(filename)
multichannel = tifffile.imread(multichannel_path)
z, c, y, x = multichannel.shape
assert c == min(z, c, y, x)

for i in range(c):
    channel = multichannel[:, i, :, :]
    dst = os.path.join(folder, f"{base}_c{i}{ext}")
    tifffile.imwrite(dst, channel, imagej = True, metadata = {'axes': 'ZYX'})

C:\Users\bear\AppData\Local\Programs\Python\Python312\Lib\site-packages\tifffile\tifffile.py:3784: UserWarning: <tifffile.TiffWriter 'Mark_and_Find 0…sition003_c0.tif'> truncating ImageJ file
  warnings.warn(
C:\Users\bear\AppData\Local\Programs\Python\Python312\Lib\site-packages\tifffile\tifffile.py:3784: UserWarning: <tifffile.TiffWriter 'Mark_and_Find 0…sition003_c1.tif'> truncating ImageJ file
  warnings.warn(
